In [ ]:
import pandas as pd

In [10]:
file_path = 'retail_sales_dataset.csv'
data = pd.read_csv(file_path)


data = data.drop(columns=['Transaction ID', 'Date'])


In [11]:
data.head()

,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,CUST001,Male,34,Beauty,3,50,150
1,CUST002,Female,26,Clothing,2,500,1000
2,CUST003,Male,50,Electronics,1,30,30
3,CUST004,Male,37,Clothing,1,500,500
4,CUST005,Male,30,Beauty,2,50,100


In [12]:
#checking for null values
data.isnull().sum()

Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64

**Hypothesis testing**

In [9]:

from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_ind, chi2_contingency, f_oneway, pearsonr




label_encoders = {}
for column in ['Customer ID', 'Gender', 'Product Category']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# T-test: Compare Total Amount between Male and Female customers
male_total = data[data['Gender'] == 1]['Total Amount']
female_total = data[data['Gender'] == 0]['Total Amount']
t_stat, p_value = ttest_ind(male_total, female_total)
print(f'T-test: t-statistic = {t_stat}, p-value = {p_value}')

# Chi-square test: Check independence between Gender and Product Category
contingency_table = pd.crosstab(data['Gender'], data['Product Category'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
print(f'Chi-square test: chi2 = {chi2}, p-value = {p}')

# ANOVA: Compare Total Amount across different Product Categories
categories = data['Product Category'].unique()
category_data = [data[data['Product Category'] == category]['Total Amount'] for category in categories]
f_stat, p_value = f_oneway(*category_data)
print(f'ANOVA: F-statistic = {f_stat}, p-value = {p_value}')

# Correlation: Check correlation between Age and Total Amount
corr, p_value = pearsonr(data['Age'], data['Total Amount'])
print(f'Correlation: correlation coefficient = {corr}, p-value = {p_value}')


T-test: t-statistic = -0.03161341824319852, p-value = 0.9747866634918314
Chi-square test: chi2 = 1.673837085800602, p-value = 0.43304287262068974
ANOVA: F-statistic = 0.159430595139859, p-value = 0.8526508750301867
Correlation: correlation coefficient = -0.06056802388304562, p-value = 0.05553184763493584


Interpretation of Results:

**T-test:** A low p-value (typically < 0.05) indicates a significant difference in the means of Total Amount spent by Male and Female customers.
**Chi-square test**: A low p-value suggests a significant association between Gender and Product Category.
**ANOVA:** A low p-value indicates significant differences in the means of Total Amount across different Product Categories.
**Correlation**: A high absolute value of the correlation coefficient (close to 1 or -1) and a low p-value suggest a strong correlation between Age and Total Amount.

****

**Pycaret**

PyCaret is an open-source, low-code machine learning library in Python that automates machine learning workflows. It is designed to reduce the time required to develop and deploy machine learning models. PyCaret is particularly useful for data scientists and analysts who want to perform complex machine learning tasks with minimal coding

In [1]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.

In [4]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pycaret.regression import setup, compare_models, predict_model



# Encode categorical variables
label_encoders = {}
for column in ['Customer ID', 'Gender', 'Product Category']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Separate features and target
X = data.drop(columns=['Total Amount'])
y = data['Total Amount']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the PyCaret regression setup
reg = setup(data=pd.concat([X_train, y_train], axis=1), target='Total Amount', verbose=False)

# Compare different models and find the best one
best_model = compare_models()

# Evaluate the best model on the test set
predictions = predict_model(best_model, data=X_test)
print(predictions)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0250
et,Extra Trees Regressor,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.1360
gbr,Gradient Boosting Regressor,1.1169,3.1301,1.6508,1.0000,0.0131,0.0085,0.1120
lightgbm,Light Gradient Boosting Machine,1.0775,11.9811,2.6604,1.0000,0.0192,0.0073,0.2610
xgboost,Extreme Gradient Boosting,0.0001,0.0000,0.0002,1.0000,0.0000,0.0000,0.0690
rf,Random Forest Regressor,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.2030
ada,AdaBoost Regressor,77.8089,13248.1689,114.3382,0.9584,0.2402,0.2255,0.0440
lasso,Lasso Regression,177.9941,47028.7770,216.2009,0.8519,0.9534,1.6862,0.0250
llar,Lasso Least Angle Regression,177.9941,47028.7758,216.2009,0.8519,0.9534,1.6862,0.0380
br,Bayesian Ridge,177.7935,47126.2343,216.4123,0.8516,0.9384,1.6724,0.0600


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

     Customer ID  Gender  Age  Product Category  Quantity  Price per Unit  \
521          522       1   46                 0         3             500   
737          738       1   41                 1         2              50   
740          741       1   48                 1         1             300   
660          661       0   44                 1         4              25   
411          412       0   19                 2         4             500   
..           ...     ...  ...               ...       ...             ...   
408          409       0   21                 2         3             300   
332          333       0   54                 2         4             300   
208          209       0   30                 2         4              50   
613          614       0   39                 0         4             300   
78            78       1   34                 0         1             300   

     prediction_label  
521            1500.0  
737             100.0  
740

**AutoMl library -tpot**

In [5]:
pip install tpot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=d1f6477161421803d96ec931ee6b733f64adfb8adfa4fa91d4aa1eeb71891866
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [8]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tpot import TPOTRegressor

label_encoders = {}
for column in ['Customer ID', 'Gender', 'Product Category']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


X = data.drop(columns=['Total Amount'])
y = data['Total Amount']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


tpot = TPOTRegressor(verbosity=2, generations=5, population_size=50, random_state=42)


tpot.fit(X_train, y_train)


print(tpot.score(X_test, y_test))

# Export the pipeline
tpot.export('tpot_best_pipeline.py')


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.0

Generation 2 - Current best internal CV score: 0.0

Generation 3 - Current best internal CV score: 0.0

Generation 4 - Current best internal CV score: 0.0

Generation 5 - Current best internal CV score: 0.0

Best pipeline: DecisionTreeRegressor(input_matrix, max_depth=6, min_samples_leaf=17, min_samples_split=5)
-0.0
